In [1]:
import pandas as pd
import producer

pd.set_option('display.max_colwidth', None)

In [2]:
log_file = "assets/log_file.log"
input_file = "assets/processed_log.csv"
processed_file = "assets/log_data.csv"

input_df = pd.read_csv(input_file)
input_df

,timestamp,code,message,ip_address,label
0,Dec 10 06:55:46,sshd[24200],reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT!,173.234.31.186,break_in_attempt
1,Dec 10 06:55:46,sshd[24200],Invalid user webmaster from 173.234.31.186,173.234.31.186,no label
2,Dec 10 06:55:46,sshd[24200],input_userauth_request: invalid user webmaster [preauth],0,invalid_user
3,Dec 10 06:55:46,sshd[24200],pam_unix(sshd:auth): check pass; user unknown,0,auth_failure
4,Dec 10 06:55:46,sshd[24200],pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186,173.234.31.186,auth_failure
...,...,...,...,...,...
1995,Dec 10 11:04:42,sshd[25539],pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=103.99.0.122,103.99.0.122,auth_failure
1996,Dec 10 11:04:43,sshd[25541],Failed password for root from 183.62.140.253 port 36300 ssh2,183.62.140.253,failed_password
1997,Dec 10 11:04:43,sshd[25541],Received disconnect from 183.62.140.253: 11: Bye Bye [preauth],183.62.140.253,disconnect
1998,Dec 10 11:04:43,sshd[25544],pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=183.62.140.253 user=root,183.62.140.253,auth_failure


In [3]:
lines = producer.to_dict(log_file)
# print(lines)
df = pd.DataFrame(lines)

In [4]:
df.head()

,timestamp,process_id,username,ip,is_private,is_root,message,label
0,1.670673e+09,24200,,173.234.31.186,0,0,reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT!,break_in_attempt
1,1.670673e+09,24200,webmaster,173.234.31.186,0,0,Invalid user webmaster from 173.234.31.186,no label
2,1.670673e+09,24200,,173.234.31.186,0,0,pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186,auth_failure
3,1.670673e+09,24200,webmaster,173.234.31.186,0,0,Failed password for invalid user webmaster from 173.234.31.186 port 38926 ssh2,failed_password
4,1.670673e+09,24200,,173.234.31.186,0,0,Connection closed by 173.234.31.186 [preauth],connection_closed


In [16]:
pro_lines = []
index = 0

while index < len(lines):
    # Check if line 1 and line 2 have the same IP Address. Usernames might be different, so we cant be sure on that.
    # Merge rows only when their timestamps match
    line1 = lines[index]
    
    # ignore some lines
    if line1['label'] in ["connection_closed", "disconnect", "message_repeated"]:
        index += 1
        continue
    
    if line1['label'] == "no label":
        index += 1
        continue
    
    if index + 1 < len(lines):
        line2 = lines[index + 1]
        if (line1['timestamp'] == line2['timestamp']) and (line1['ip'] == line2['ip']):
            line1_message, line1_label = line1['message'], line1["label"]
            line2_message, line2_label = line2['message'], line2["label"]

            # define various cases for merge:
            if line1_label == "break_in_attempt" and line2_label == "no label":
                line1["username"] = line2["username"] if not line1["username"] else line1["username"]
                line1["message"] = f"{line1['message']} -- {line2['message']}"

                index += 1
            
            # elif line1_label == "auth_failure" and line2_label == "failed_password":

    pro_lines.append(line1)
    index += 1

In [17]:
print(f"Original Lines: {len(lines)}, After Compression: {len(pro_lines)}")

Original Lines: 1734, After Compression: 1114


In [21]:
prodf = pd.DataFrame(pro_lines)
prodf.head(50)

,timestamp,process_id,username,ip,is_private,is_root,message,label
0,1.670673e+09,24200,webmaster,173.234.31.186,0,0,reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT! -- Invalid user webmaster from 173.234.31.186 -- Invalid user webmaster from 173.234.31.186 -- Invalid user webmaster from 173.234.31.186,break_in_attempt
1,1.670673e+09,24200,,173.234.31.186,0,0,pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186,auth_failure
2,1.670673e+09,24200,webmaster,173.234.31.186,0,0,Failed password for invalid user webmaster from 173.234.31.186 port 38926 ssh2,failed_password
3,1.670674e+09,24206,test9,52.80.34.196,0,0,Failed password for invalid user test9 from 52.80.34.196 port 36060 ssh2,failed_password
4,1.670674e+09,24208,webmaster,173.234.31.186,0,0,reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT! -- Invalid user webmaster from 173.234.31.186 -- Invalid user webmaster from 173.234.31.186 -- Invalid user webmaster from 173.234.31.186,break_in_attempt
5,1.670674e+09,24208,,173.234.31.186,0,0,pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186,auth_failure
6,1.670674e+09,24208,webmaster,173.234.31.186,0,0,Failed password for invalid user webmaster from 173.234.31.186 port 39257 ssh2,failed_password
7,1.670674e+09,24224,,202.100.179.208,0,0,pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=202.100.179.208,auth_failure
8,1.670674e+09,24224,chen,202.100.179.208,0,0,Failed password for invalid user chen from 202.100.179.208 port 32484 ssh2,failed_password
9,1.670674e+09,24227,root,5.36.59.76,0,1,pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=5.36.59.76.dynamic-dsl-ip.omantel.net.om user=root,auth_failure


In [19]:
prodf['label'].unique()

array(['break_in_attempt', 'auth_failure', 'failed_password',
       'no_identification'], dtype=object)

In [ ]:
prodf.to_csv(processed_file, index=False)